Resources:
- [Medium blog post](https://medium.com/@tejpal.abhyuday/optimizing-language-model-fine-tuning-with-peft-qlora-integration-and-training-time-reduction-04df39dca72b)

In [3]:
!pip install -q -U bitsandbytes peft accelerate einops